# ELAN to TEI conversion 

**Author:** Daniel Schopper    
**Description:** This notebook automates the ELAN to TEI conversion in the SHAWI Project. When finished, it should be ported to a non-interactive script which is triggered by a github action.    
**Last Change:** 2022-03-23     
**History:**    
* 2022-03-10: Initital set up
* 2022-03-11: added XSLT transformation via saxonpy
* 2022-03-12: added ELAN to TEI conversion
* 2022-03-23: integrated merge metadata XSL

## TODOS and Open Questions

* merge documents, making sure to not overwrite manual changes
* replace TEI documents with x-includes
* tokenize
* validate

In [16]:
import io
import sharepy
import os
import requests
import pathlib
#import filetype – not used
from pathlib import Path
from urllib.parse import urlsplit
from saxonpy import PySaxonProcessor, PyXdmValue
from zipfile import ZipFile
import subprocess
from datetime import datetime
# from inspect import getmembers, signature

## Configuration

In [58]:
# the URL of the Sharepoint installation 
sp_baseURL = "oeawacat.sharepoint.com"

# the sharepoint username + password are taken from the environment
sp_username = os.environ['SP_USERNAME']
sp_pwd = pwd = os.environ['SP_PWD']

# the name of the Sharepoint Site
sp_siteName = "ACDH-CH_p_ShawiTypeArabicDialects_Shawi"

# the path to the Excel file
sp_pathToRecordingsXLSX = "Shared Documents/General/Shawi_Recordings.xlsx"


# the name of the local directory where downloaded data will be stored
dataDir = "data"

# the name of the local directory where downloaded libraries and other auxiliary code will be stored
libDir = "lib"

# the root of the git repository
shawidataHomeDir = "../.."

# path to project-specific stylesheets
pathToShawiStylesheetsDir = shawidataHomeDir+"/082_scripts_xsl"

# the path to the ELAN transcription files
pathToELANDir = shawidataHomeDir+"/122_elan"

# the path to the TEI transcription files
pathToTEIDir = shawidataHomeDir+"/102_derived_tei"

# the path to the NoSkE verticals
noSkEVertDir = shawidataHomeDir+"/131_vert_xml"

# the path to the tei Corpus document produced by this script
pathToTeiCorpus = pathToTEIDir+"/shawiCorpus.xml"


# the path to the audio files
pathToRecordingsDir = "THIS_IS_NOT_USED"#"/mnt/univie_orientalistik/SHAWI/Recordings"

proc = PySaxonProcessor(license=False)
# SaxonC 1.2.1 Python has many known bugs but isn't maintained anymore
# Many of the documented API specs are not working
print(proc.version)
proc.set_cwd(os.path.dirname(os.path.abspath('')))
print(proc.cwd)


#set up directories
print("** setting up directories **")
for i in [dataDir,libDir]: 
    if os.path.exists(i):
        print("skipped existing directory '"+i+"'")
    else:
        os.mkdir(i)
        print("created directory '"+i+"'")
        
        
# define which steps should be skipped. 

SKIP_PROCESSING = []#["runTEICorpo"]

Saxon/C 1.2.1 running with Saxon-HE 9.9.1.5C from Saxonica
/home/simar/shawi-data/080_scripts_generic
** setting up directories **
skipped existing directory 'data'
skipped existing directory 'lib'


## Setup

### Step 1: get the latest release of the TEI Stylesheets 

In [18]:
# Setup

# fetch the TEI Stylesheets    
def installFromGithub(libraryName):
    headers = {"Accept" : "application/vnd.github.v3+json"}
    repo = libraryName
    print("** Fetching library "+repo+" **")
    libBasePath = libDir+"/"+repo
    
    # First we check which tag name the latest release has
    r = requests.get("https://api.github.com/repos/"+repo+"/releases/latest", headers=headers)
    if r.status_code != 200:
        print("An error occured fetching the latest release. Maybe there isn't any release? ")
        print(r.content)
        return 1
    release = r.json()
    tag = release["tag_name"]
    
    # we check whether we have the latest version already \
    # by checking if the respective path is already installed
    libReleasePath = libBasePath+"/"+tag
    haveLatestVersion = os.path.exists(libReleasePath)
    if haveLatestVersion:
        print("We have already the latest version ("+tag+"). Exiting")
        print("")
        return libReleasePath
    else:
        url = release["assets"][0]["browser_download_url"]
        payload = requests.get(url).content
        zipfilename = os.path.basename(url)
        os.makedirs(libReleasePath, exist_ok=True)
        zipfilePath = libReleasePath +"/"+zipfilename
        open(zipfilePath, 'wb').write(payload)
        ZipFile(zipfilePath).extractall(path=libReleasePath)
        print("Downloaded latest version ("+tag+") to "+libReleasePath)
        print("")
        return libReleasePath


pathToTEIGuidelines=installFromGithub("TEIC/TEI")
pathToTEIStylesheets=installFromGithub("TEIC/Stylesheets")


** Fetching library TEIC/TEI **
We have already the latest version (P5_Release_4.4.0). Exiting

** Fetching library TEIC/Stylesheets **
We have already the latest version (v7.53.0). Exiting



### Step 2: Download the latest version of the Excel Sheet

In [19]:
# TODO will need to add credentials if this is run in non-interactive mode
def downloadFromSP(sp_filepath, force=False):
    url = "https://"+sp_baseURL+"/sites/"+sp_siteName+"/"+sp_filepath
    print("attempting to download file from "+url)
    filename = os.path.basename(sp_filepath)
    downloadPath = dataDir+"/"+filename
    if os.path.exists(downloadPath) and not force:
        print("skipping existing file "+downloadPath)
        return downloadPath
    else:
        s = sharepy.connect(sp_baseURL, username=sp_username, password=sp_pwd)
        s.getfile(url, filename=downloadPath)
        return downloadPath


pathToExcelSheet = downloadFromSP(sp_pathToRecordingsXLSX, force="downloadExcelSheet" not in SKIP_PROCESSING)
print(pathToExcelSheet)

attempting to download file from https://oeawacat.sharepoint.com/sites/ACDH-CH_p_ShawiTypeArabicDialects_Shawi/Shared Documents/General/Shawi_Recordings.xlsx
data/Shawi_Recordings.xlsx


## Step 2: transform xlsx to TEI table

In [20]:
def transform(s, xsl, o, parameters=[]):
    # no change in behavior
    proc.set_configuration_property("xi", "on")
    saxon = proc.new_xslt_processor()
    saxon.set_source(file_name=os.path.abspath(s))
    for i in parameters:
        saxon.set_parameter(name=i, value=proc.make_string_value(parameters[i]))
    saxon.compile_stylesheet(stylesheet_file=os.path.abspath(xsl))
    saxon.set_output_file(os.path.abspath(o))
    saxon.transform_to_string()
    if saxon.exception_occurred():
        # print(getmembers(saxon))
        for i in range(saxon.exception_count()-1):
            print(saxon.get_error_message(0))
        print(os.path.abspath(s)+" - "+os.path.abspath(xsl)+" -> "+os.path.abspath(o)+" failed")
    return o # for chaining

In [21]:
def xlsx2teitable(xlsx, output):

    # first, extract contents of XLSX document to a temp directory
    unzipPath=xlsx.replace(".xlsx","")
    os.makedirs(unzipPath, exist_ok=True)
    ZipFile(xlsx).extractall(path=unzipPath)
    
    # then transform the .rels file using the TEIC Stylesheets 
    pathToXlsxtoteiXSL=pathToTEIStylesheets+"/xml/tei/stylesheet/xlsx/xlsxtotei.xsl"

    params = {
        "inputDir" : pathlib.Path(os.path.abspath(unzipPath)).as_uri(),
        "workDir" : pathlib.Path(os.path.abspath(unzipPath)).as_uri()
        
    }

    transform(
        s = unzipPath+"/_rels/.rels", 
        xsl = pathToXlsxtoteiXSL, 
        o = output, 
        parameters=params
    )
    return

In [22]:
pathToTEItable=pathToExcelSheet.replace(".xlsx",".xml")

if not "xlsx2teitable" in SKIP_PROCESSING:    
    xlsx2teitable(xlsx=pathToExcelSheet, output=pathToTEItable)
    debugstring="""<!-- 
   THIS FILE IS INCLUDED IN THE GIT REPOSITORY ONLY FOR DEBUGGING PURPOSES. 
   
   The source of this file is constantly being edited at 
   https://oeawacat.sharepoint.com/sites/ACDH-CH_p_ShawiTypeArabicDialects_Shawi/_layouts/15/Doc.aspx?sourcedoc={F01FF43B-2409-4E31-A5BF-653E0559B160}&file=SHAWI%20Recordings.xlsx&action=default&mobileredirect=true&cid=f7311564-c2b6-4b08-9a52-468547688408
   So this copy is most probably already outdated.
   
  To update it, you can either run https://gitlab.com/acdh-oeaw/shawibarab/shawi-data/-/blob/main/080_scripts_generic/080_01_ELAN2TEI/ELAN2TEI.ipyn
   *OR*  
   1) download the Excel file manually from Sharepoint
   2) and tranform it to TEI using oxgarage.tei-c.org/ 
   
-->
    """
    f = open(pathToTEItable,mode="r",encoding="UTF8")
    src = f.read()
    new = src.replace('<?xml version="1.0" encoding="UTF-8"?>','<?xml version="1.0" encoding="UTF-8"?>\n'+debugstring)
    f.close()
    f = open(pathToTEItable, mode="wt",encoding="UTF8")
    f.write(new)
    f.close()
        
    print(pathToTEItable)

data/Shawi_Recordings.xml


## transform TEI table to corpus header

In [23]:
pathToTeitableToCorpusXSL=pathToShawiStylesheetsDir+"/table2corpus.xsl"
params = {
    "pathToRecordings" : pathlib.Path(os.path.abspath(pathToRecordingsDir)).as_uri()
}
try:
    transform(pathToTEItable, pathToTeitableToCorpusXSL, pathToTeiCorpus, params)
except:
    print("an error occured")
print(pathToTeiCorpus)

../../102_derived_tei/shawiCorpus.xml


## Run TEICorpo

In [24]:
def installFromUrl(url, force=False):
    r = requests.get(url)
    filename = os.path.basename(urlsplit(url).path)
    downloadpath = libDir+"/"+filename
    if os.path.exists(downloadpath) and not force:
        print("skipping download")
    else:
        open(downloadpath, 'wb').write(r.content)
        print("file "+downloadpath+" downloaded")
    return downloadpath

# TODO check for filetype and automatically extract zip file 
# so this can be re-used for the insta
 
installFromUrl("https://github.com/christopheparisse/teicorpo/blob/e06a01ad5cb4c3aef631b3749ce59b5eb6f5ea11/teicorpo.jar?raw=true")
installFromUrl("https://repo1.maven.org/maven2/commons-io/commons-io/2.11.0/commons-io-2.11.0.jar")
pathToTeiCorpo=libDir+"/*"
print(pathToTeiCorpo)

skipping download
skipping download
lib/*


Collect all ELAN documents from pathToELANDir

In [25]:
ELANDocs = []

for i in os.scandir(pathToELANDir):
    if os.path.basename(i).endswith(".eaf"):
        ELANDocs.append({
            "filepath" : os.path.abspath(i),
            "filename" : os.path.basename(i),
            "basename" : Path(i).stem
        })
for d in ELANDocs:
    print(d["filepath"])

/home/simar/shawi-data/122_elan/Urfa-012_Lentils-Harran-2010.eaf
/home/simar/shawi-data/122_elan/Urfa-077_Jinn-Harran-2010.eaf


In [38]:
def runTEICorpo(docs = dict):
    for i in docs:
        pathToInput = i["filepath"]
        filenameELAN = i["filename"]
        runtime = datetime.now().strftime("%Y-%m-%d_%H-%M")
        tmpDir = pathToTEIDir+"/"+runtime
        filenameTEI = i["basename"]+".xml"
        pathToOutput = tmpDir+"/"+"ELAN_"+filenameTEI
        os.makedirs(tmpDir, exist_ok=True)
        i["filepath_tmp_TEI"] = os.path.abspath(pathToOutput)
        i["tmpDir"] = tmpDir
        output = os.path.abspath(pathToTEIDir + "/" + i["basename"] + ".xml")
        i["TEI"] = os.path.abspath(output)
        res = subprocess.run(["java", "-cp", pathToTeiCorpo, "-Dfile.encoding=UTF-8", "fr.ortolang.teicorpo.TeiCorpo", "-from","elan", "-to","tei", "-o",pathToOutput, pathToInput], capture_output=True, encoding="UTF-8")
        print(res.stdout)
        print(res.stderr)
        print(pathToOutput)

run TEI Corpo on all ELANDocs, writing the path to the TEI output back to the variable

In [49]:
if not "runTEICorpo" in SKIP_PROCESSING:
    runTEICorpo(docs=ELANDocs)

TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../102_derived_tei/2022-07-22_21-22/ELAN_Urfa-012_Lentils-Harran-2010.xml
TeiCorpo (version 1.4.44) 14/12/2021 08:07 Version TEI_CORPO: 0.9.1


../../102_derived_tei/2022-07-22_21-22/ELAN_Urfa-077_Jinn-Harran-2010.xml


### Prepare

Merge metadata and TEICorpo output:

In [50]:
def mergeMetadata(docInfo, p):
    return transform(
        s = docInfo["filepath_tmp_TEI"],
        xsl = pathToShawiStylesheetsDir+"/mergeHeaderAndTranscription.xsl",
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_00_metaMerged.xml",
        parameters = p)

## Tokenize

Run a local copy of [xsl-tokenizer](https://github.com/acdh-oeaw/xsl-tokenizer)

### Step 0 (optional)

Regenerate the XSLs used in the following steps.
This can not be done with saxonpy (xincludes are not resolved)
use
```bash
java -jar Saxon-HE-9.9.1-8.jar -s:profile.xml -xi:on -xsl:xsl/make_xsl.xsl
```

For all the ELAN files converted to TEI:

### Step 1

Remove new lines and store to intermediate document:

In [51]:
def removeNL(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_00_metaMerged.xml", 
        xsl = "./tokenizer/xsl/rmNl.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_01_nlRmd.xml"
    )

### Step 2

Tokenize:

In [52]:
def tokenize(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_01_nlRmd.xml", 
        xsl = "./tokenizer/wrapper_toks.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_02_toks.xml"
    )

### Step 3

Add Part-Attributes and explicit token links:

In [53]:
def addP(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_02_toks.xml", 
        xsl = "./tokenizer/wrapper_addP.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_03_tokenized.xml"
    )

### Step 4

Do some post tokenization processing specific to the Shawi project:

In [54]:
def postProcess(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_03_tokenized.xml", 
        xsl = "./tokenizer/postTokenization/1.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_04_posttok.xml"
    )

### Step 5

Create a intermedieate vertical document that has a structure suited for further processing to TEI and to NoSkE verticals

In [61]:
def createVertSource(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_04_posttok.xml", 
        xsl = "./tokenizer/wrapper_xtoks2vert.xsl", 
        o = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_05_vert.xml"
    )

### Step 6

Create the final tokenized TEI document

In [64]:
def createFinalTEI(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_05_vert.xml",
        xsl = "./tokenizer/xsl/xtoks2tei.xsl", 
        o = docInfo["TEI"],
        parameters = {
            "preserve-ws": "false"
        }
    )

### Step 7

Create a vertical vor NoSkE

In [65]:
def createNoSkEVert(docInfo):
    return transform(
        s = docInfo["tmpDir"]+'/'+docInfo["basename"]+"_05_vert.xml",
        xsl = "./tokenizer/wrapper_vert2txt.xsl", 
        o = noSkEVertDir + "/" + docInfo["basename"] + ".xml"
    )

In [66]:
print(os.path.abspath(pathToTeiCorpus))
mergeParam = { "pathToCorpusDoc": pathlib.Path(os.path.abspath(pathToTeiCorpus)).as_uri() }
for doc in ELANDocs:
    print(doc["basename"]+': '+doc["filepath_tmp_TEI"]+" -> "+doc["TEI"])
    mergeMetadata(doc, mergeParam)
    removeNL(doc)
    tokenize(doc)
    addP(doc)
    postProcess(doc)
    createVertSource(doc)
    createFinalTEI(doc)
    createNoSkEVert(doc)
    print(doc["basename"]+": done.")

/home/simar/shawi-data/102_derived_tei/shawiCorpus.xml
Urfa-012_Lentils-Harran-2010: /home/simar/shawi-data/102_derived_tei/2022-07-22_21-22/ELAN_Urfa-012_Lentils-Harran-2010.xml -> /home/simar/shawi-data/102_derived_tei/Urfa-077_Jinn-Harran-2010.xml
Urfa-012_Lentils-Harran-2010: done.
Urfa-077_Jinn-Harran-2010: /home/simar/shawi-data/102_derived_tei/2022-07-22_21-22/ELAN_Urfa-077_Jinn-Harran-2010.xml -> /home/simar/shawi-data/102_derived_tei/Urfa-077_Jinn-Harran-2010.xml


addP.xsl
xtoks2vert.xsl
Debug level: false


Urfa-077_Jinn-Harran-2010: done.


addP.xsl
xtoks2vert.xsl
Debug level: false


## Replace TEI elements with x-includes in corpus document